In [3]:
from math import floor, ceil
from statistics import mean
from scipy.optimize import brentq
import numpy as np

In [4]:
race_length = 8 * 3600  # s
long_stop_time = 180  # s
long_stops = 3
pitlane_drive_through_time = 18.5

In [5]:
def estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint = None):  # TODO multi-driver support, optional parameter for time remaining in the race
    global race_length
    global long_stops
    global long_stop_time
    global base_pitstop_loss

    global fuel_tank_size
    global refuel_rate
    global tire_swap_time

    total_long_stop_time = long_stops * long_stop_time
    effective_race_length = race_length - total_long_stop_time + 2 * average_lap_time  # Handling long stops like this sucks but it'll work until this function gets deprecated

    if laps_per_stint:
        liters_to_refuel = laps_per_stint * average_fuel_consumption
    else:
        liters_to_refuel = fuel_tank_size - average_fuel_consumption  # To get a lower bound on max_stint_length, we lower bound pitstop length by
        # assuming there is at least one lap of fuel left in the tank when coming in (TODO decide if this is the right thing to do)

    pitstop_length = base_pitstop_loss + max(tire_swap_time, refuel_rate * liters_to_refuel)

    if not laps_per_stint:
        laps_per_stint = floor(fuel_tank_size / average_fuel_consumption)  # TODO Can this be moved up to avoid having the previous if laps_per_stint block?
    
    max_stint_length = average_lap_time * laps_per_stint + pitstop_length
    # TODO: Parameter for max laps out? (tire strategy, may be better to save for new strat calculator)

    stint_total = (effective_race_length + average_lap_time) / max_stint_length  # Adding average_lap_time to numerator accounts for 6 hours + 1 lap
    # We want an upper bound of this
    # PROBLEM! Does not account for the fact that long stops have free pit stop!!!  Long stops are removed from numerator but also need to be removed from denominator
    # To fix, max_stint_length should be a weighted average of max_stint_length
    average_stint_length = average_lap_time * laps_per_stint + ((stint_total - long_stops) / stint_total) * pitstop_length
    new_stint_total = (effective_race_length + average_lap_time) / average_stint_length  # TODO make this a loop that breaks once it converges well enough

    # pitstop_length is the sum of the time lost between the inlap and outlap. The first stint does not have an outlap but rather a formation lap (which is longer than an outlap),
    # the final stint has an outlap but not an inlap. Therefore right now stint_total is overestimated by outlap_differential / max_stint_length

    seconds_margin = (round(new_stint_total) - new_stint_total) * average_stint_length
    
    return new_stint_total, max_stint_length, seconds_margin  # Separate function for max_stint_length?



In [6]:
def laps_and_fuel_per_stint(average_lap_time, average_fuel_consumption, num_pitstops):  # TODO: Consider the fact that long stops should refuel more than short stops
    global fuel_tank_size
    stint_total = num_pitstops + 1
    obj = lambda laps_per_stint: estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint)[0] - stint_total

    # Calculate optimal laps per stint
    optimal_laps_per_stint = brentq(obj, 1, floor(fuel_tank_size / average_fuel_consumption))

    # Return liters required to do optimal laps
    return ceil(optimal_laps_per_stint), ceil(average_fuel_consumption * ceil(optimal_laps_per_stint)), optimal_laps_per_stint, average_fuel_consumption * optimal_laps_per_stint

In [7]:
def pit_time_matrix(average_lap_time, average_fuel_consumption, num_pitstops):  # Number of completed stints on axis 0, number of completed long stops on axis 1
    # Current method works from the back forward, a "worst case scenario" approach should go back to front
    global long_stops
    global long_stop_time
    global race_length
    global base_pitstop_loss
    global tire_swap_time
    global refuel_rate

    laps_per_stint, liters_to_refuel, _, _ = laps_and_fuel_per_stint(average_lap_time, average_fuel_consumption, num_pitstops)  # Maybe use soft numbers for laps and fuel per stint?
    _, stint_length, _ = estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint)
    pitstop_length = base_pitstop_loss + max(tire_swap_time, refuel_rate * liters_to_refuel)

    # Create 1D arrays
    long_stop_array = np.arange(long_stops + 1).reshape((1, -1))
    pit_stops_array_base = np.arange(1, num_pitstops + 1).reshape((-1, 1))  # First element is before first pitstop (after first stint), last element is final pitstop

    # Convert pit_stops_array to times that car should enter pits if there are no long stops
    pit_stops_array = race_length - (pit_stops_array_base * stint_length) + pitstop_length  # Adding back pitstop_length is necessary since the pitstop hasn't been done on entry
    pit_stops_array -= 2 * average_lap_time  # Subtracting average_lap_time sets the end of the final stint to be one lap after the timer hits 0, ensuring there's enough fuel to get to the end
    # TODO figure out if this messes with first stint by "extending" it (it probably does, should not be used this way)
    pit_stops_array = pit_stops_array.astype(np.int32)

    # Factor in long stops
    pit_stops_matrix = (pit_stops_array - long_stop_time * long_stop_array)

    # Convert array to timedelta
    pit_stops_matrix_seconds = (pit_stops_matrix % 60).astype(np.str_)
    pit_stops_matrix_minutes = ((pit_stops_matrix // 60) % 60).astype(np.str_)
    pit_stops_matrix_hours = (pit_stops_matrix // 3600).astype(np.str_)
    pit_stops_matrix_string = pit_stops_matrix_hours + ":" + pit_stops_matrix_minutes + ":" + pit_stops_matrix_seconds  # Consider printing strings line by line

    # Replace invalid elements of array with NaN (fewer long stops than stops completed and more long stops than pit stops remaining)
    pit_stops_matrix_string[(pit_stops_array_base <= long_stop_array) | (num_pitstops - pit_stops_array_base < long_stops - long_stop_array)] = "NaN"

    return pit_stops_matrix_string

In [14]:
def pit_time_matrix_reverse(average_lap_time, average_fuel_consumption, num_pitstops):
    global long_stops
    global long_stop_time
    global race_length
    global base_pitstop_loss
    global tire_swap_time
    global refuel_rate

    laps_per_stint, _, _, _ = laps_and_fuel_per_stint(average_lap_time, average_fuel_consumption, num_pitstops)  # Maybe use soft numbers for laps and fuel per stint?
    _, stint_length, _ = estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint)

    # Create 1D arrays
    long_stop_array = np.arange(long_stops + 1).reshape((1, -1))[::-1]
    pit_stops_array_base = np.arange(1, num_pitstops + 1).reshape((-1, 1))[::-1]  # First element is before first pitstop (after first stint), last element is final pitstop

    # Convert pit_stops_array to times that car should enter pits if there are no long stops
    pit_stops_array = pit_stops_array_base * stint_length
    pit_stops_array -= 2 * average_lap_time  # Subtracting average_lap_time sets the end of the final stint to be one lap after the timer hits 0, ensuring there's enough fuel to get to the end
    pit_stops_array = pit_stops_array.astype(np.int32)

    # Factor in long stops
    pit_stops_matrix = (pit_stops_array + long_stop_time * long_stop_array)

    # Convert array to timedelta
    pit_stops_matrix_seconds = (pit_stops_matrix % 60).astype(np.str_)
    pit_stops_matrix_minutes = ((pit_stops_matrix // 60) % 60).astype(np.str_)
    pit_stops_matrix_hours = (pit_stops_matrix // 3600).astype(np.str_)
    pit_stops_matrix_string = pit_stops_matrix_hours + ":" + pit_stops_matrix_minutes + ":" + pit_stops_matrix_seconds  # Consider printing strings line by line

    # Replace invalid elements of array with NaN (fewer long stops than stops completed and more long stops than pit stops remaining)
    alt_pit_stops_array_base = np.arange(1, num_pitstops + 1).reshape((-1, 1))
    alt_long_stop_array = np.arange(long_stops + 1).reshape((1, -1))
    pit_stops_matrix_string[(alt_pit_stops_array_base <= alt_long_stop_array) | (num_pitstops - alt_pit_stops_array_base < long_stops - alt_long_stop_array)] = "NaN"

    return pit_stops_matrix_string

In [9]:
def pit_time_matrix_simple(num_pitstops, average_lap_time):  # Average lap time is only used for building in assumption that race ends 1 lap after timer hits 0
    '''
    This version just takes the number of pitstops and spreads stints into even intervals to act as a rough reference for whether the race can be done on that number of stops.
    If sufficiently ahead of the pit entry window, drivers can opt to end the stint early or shorten the next stint.
    If behind the pit window, drivers will need to extend their stints or switch to another strategy.
    '''  # Idea: calculating total buffer?
    global long_stops
    global long_stop_time
    global race_length
    global base_pitstop_loss
    global tire_swap_time
    global refuel_rate

    # Create 1D arrays
    long_stop_array = np.arange(long_stops + 1)[::-1].reshape((1, -1))
    pit_stops_array_base = np.linspace(0, 1, num = num_pitstops + 1, endpoint=False)[:0:-1].reshape((-1, 1))
    # First element is before first pitstop (after first stint), last element is final pitstop

    # Convert pit_stops_array to times that car should enter pits if there are no long stops
    pit_stops_array = (race_length + 2 * average_lap_time - long_stops * long_stop_time) * pit_stops_array_base  # TODO double check work!
    pit_stops_array -= 2 * average_lap_time  # Subtracting average_lap_time sets the end of the final stint to be one lap after the timer hits 0, ensuring there's enough fuel to get to the end
    # (2 extra laps is worst case scenario where car is in front of leader (about to be lapped) when timer hits 0 and car crosses finish line before leader takes checkered)
    # So the 2 can be lowered depending on risk tolerance
    
    pit_stops_array = pit_stops_array.astype(np.int32)

    # Factor in long stops
    pit_stops_matrix = (pit_stops_array + long_stop_time * long_stop_array)

    # Convert array to timedelta
    pit_stops_matrix_seconds = (pit_stops_matrix % 60).astype(np.str_)
    pit_stops_matrix_minutes = ((pit_stops_matrix // 60) % 60).astype(np.str_)
    pit_stops_matrix_hours = (pit_stops_matrix // 3600).astype(np.str_)
    pit_stops_matrix_string = pit_stops_matrix_hours + ":" + pit_stops_matrix_minutes + ":" + pit_stops_matrix_seconds  # Consider printing strings line by line

    # Replace invalid elements of array with NaN (fewer long stops than stops completed and more long stops than pit stops remaining)
    alt_pit_stops_array_base = np.arange(1, num_pitstops + 1).reshape((-1, 1))
    alt_long_stop_array = np.arange(long_stops + 1).reshape((1, -1))
    pit_stops_matrix_string[(alt_pit_stops_array_base <= alt_long_stop_array) | (num_pitstops - alt_pit_stops_array_base < long_stops - alt_long_stop_array)] = "NaN"

    return pit_stops_matrix_string

In [ ]:
# GT3
base_pitstop_loss = 19
# TODO: Separate losses for in, outlap? Would be messy because of finish line location varying between tracks, along with fact that some pit boxes are before finish line, some after
fuel_tank_size = 100
refuel_rate = 0.353  # sec / L
tire_swap_time = 30

In [16]:
estimate_stint_length(240.633, 7.62)
# Me

(9.161291206835543, 3180.83914, -510.24744881027164)

In [12]:
laps_and_fuel_per_stint(240.633, 7.62, 9)

(12, 92, 11.89313472049294, 90.6256865701562)

In [ ]:
pit_time_matrix_simple(9, 240.633)

array([['7:12:5', 'NaN', 'NaN', 'NaN'],
       ['6:24:11', '6:21:11', 'NaN', 'NaN'],
       ['5:36:17', '5:33:17', '5:30:17', 'NaN'],
       ['4:48:23', '4:45:23', '4:42:23', '4:39:23'],
       ['4:0:29', '3:57:29', '3:54:29', '3:51:29'],
       ['3:12:35', '3:9:35', '3:6:35', '3:3:35'],
       ['NaN', '2:21:41', '2:18:41', '2:15:41'],
       ['NaN', 'NaN', '1:30:46', '1:27:46'],
       ['NaN', 'NaN', 'NaN', '0:39:52']], dtype='<U65')

In [15]:
pit_time_matrix_reverse(240.633, 7.62, 9)

array([['7:12:57', 'NaN', 'NaN', 'NaN'],
       ['6:23:57', '6:26:57', 'NaN', 'NaN'],
       ['5:34:57', '5:37:57', '5:40:57', 'NaN'],
       ['4:45:57', '4:48:57', '4:51:57', '4:54:57'],
       ['3:56:58', '3:59:58', '4:2:58', '4:5:58'],
       ['3:7:58', '3:10:58', '3:13:58', '3:16:58'],
       ['NaN', '2:21:58', '2:24:58', '2:27:58'],
       ['NaN', 'NaN', '1:35:58', '1:38:58'],
       ['NaN', 'NaN', 'NaN', '0:49:58']], dtype='<U65')

In [71]:
estimate_stint_length(243.534, 7.76)  # Kenzie

(9.798135456012137, 2974.96872, 597.2743160100671)

In [73]:
laps_and_fuel_per_stint(243.534, 7.76, 9)

(12, 94, 11.754441482851181, 91.21446590692517)

In [121]:
# LMP2
base_pitstop_loss = 26
fuel_tank_size = 75
refuel_rate = 0.6
tire_swap_time = 10

In [75]:
estimate_stint_length(210.4, 4.6) # Marv lmp2, ~57:10

(8.471741136387829, 3434.6400000000003, -1608.7799780694465)

In [76]:
laps_and_fuel_per_stint(210.4, 4.6, 8)  # 16 is full cap

(16, 74, 15.044095926304973, 69.20284126100287)

In [123]:
pit_time_matrix_simple(8, 210.4)

array([['7:6:53', 'NaN', 'NaN', 'NaN'],
       ['6:13:46', '6:10:46', 'NaN', 'NaN'],
       ['5:20:39', '5:17:39', '5:14:39', 'NaN'],
       ['4:27:32', '4:24:32', '4:21:32', '4:18:32'],
       ['3:34:26', '3:31:26', '3:28:26', '3:25:26'],
       ['NaN', '2:38:19', '2:35:19', '2:32:19'],
       ['NaN', 'NaN', '1:42:12', '1:39:12'],
       ['NaN', 'NaN', 'NaN', '0:46:5']], dtype='<U65')

In [17]:
pit_time_matrix_reverse(210.4, 4.6, 8)

array([['7:28:30', 'NaN', 'NaN', 'NaN'],
       ['6:31:34', '6:34:34', 'NaN', 'NaN'],
       ['5:34:37', '5:37:37', '5:40:37', 'NaN'],
       ['4:37:41', '4:40:41', '4:43:41', '4:46:41'],
       ['3:40:44', '3:43:44', '3:46:44', '3:49:44'],
       ['NaN', '2:46:48', '2:49:48', '2:52:48'],
       ['NaN', 'NaN', '1:52:52', '1:55:52'],
       ['NaN', 'NaN', 'NaN', '0:58:55']], dtype='<U65')